In [75]:
import pandas as pd


In [76]:
df = pd.read_csv("Files/cards.csv")
df = df[df['language'] == 'English']
unique_names = df['name'].unique()

/tmp/ipykernel_181557/4073356801.py:1: DtypeWarning: Columns (3,7,12,16,20,23,25,26,30,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Files/cards.csv")


In [77]:
df['frameVersion'] = df['frameVersion'].replace('future', 2007)
df['frameVersion'] = df['frameVersion'].replace('1993', 1993)
df['frameVersion'] = df['frameVersion'].replace('2015', 2015)
df['frameVersion'] = df['frameVersion'].replace('2003', 2003)
df['frameVersion'] = df['frameVersion'].replace('1997', 1997)
df = df.sort_values(by='frameVersion', ascending=False)

In [78]:
intialLength = len(df)
df = df[df['isAlternative'] != True]
print(f"Removed {intialLength - len(df)} entries")

Removed 4840 entries


In [79]:
initialLength = len(df)
full_art_entries = df[df['isFullArt'] == True]

# Step 2: Check for non-full art counterparts and remove full art entries if counterparts exist
names_to_remove = full_art_entries['name'][full_art_entries['name'].isin(df[df['isFullArt'].isna()]['name'])]

# Remove the full art entries
df = df[~((df['isFullArt'] == True) & (df['name'].isin(names_to_remove)))]
print(f"Removed {intialLength - len(df)} entries")

initialLength = len(df)
df = df[~((df['setCode'] == "PLIST"))]
print(f"Removed {intialLength - len(df)} entries")

badUUIDs = set([uuid.strip() for uuid in open("URLChecker/badUUIDs.txt", "r")])
initialLength = len(df)
df = df[~df['uuid'].isin(badUUIDs)]
print(f"Removed {intialLength - len(df)} entries")

Removed 6261 entries
Removed 7588 entries
Removed 8644 entries


In [80]:
uuid_length = len("45272fb2-2ad7-5057-ae0a-8af19aca0bca")
def clean_meld_cards(df):
    def process_row(row):
        if type(row['otherFaceIds']) == str and len(row['otherFaceIds']) > uuid_length:
            row['otherFaceIds'] = ""
        return row

    return df.apply(process_row, axis=1)

In [81]:
df.loc[df['faceName'].notna(), 'name'] = df['faceName']
df = df.drop_duplicates("name")
df = clean_meld_cards(df)
name_to_find = "Brisela, Voice of Nightmares"
entry = df.loc[df['name'] == name_to_find]

print(entry["otherFaceIds"].values)

['']


In [82]:
columns_to_keep = ["name", "setCode", "number", "power", "toughness", "manaCost", "text", "type", "layout", "uuid", "otherFaceIds"]
df = df[columns_to_keep]

In [83]:
df.to_csv("cards.csv", index = False)

In [84]:
df2 = pd.read_csv("Files/cards.csv")
df2_names = set(df2['name'].unique())
df_names = set(df['name'].unique())
print(len(df2_names))
print(len(df_names))
diff = df2_names - df2_names.intersection(df_names)
diff = [x for x in diff if "//" not in x]
diff = [x for x in diff if not x.startswith("A-")]
print(diff)

/tmp/ipykernel_181557/1324849476.py:1: DtypeWarning: Columns (3,7,12,16,20,23,25,26,30,31,32,33,35,39,40,41,47,52,53,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("Files/cards.csv")


28105
28527
['Velican Dragon', 'Goblin Bruiser', "Ashuza's Breath", "Hapato's Might", 'Camato Scout', 'Blinding Radiance', 'Lydari Druid', 'Lydari Elephant', 'Titanic Pelagosaur', 'Murgish Cemetery', "Saji's Torrent", 'Tornellan Protector', 'Ogre Painbringer']
